In [2]:
import pickle 
import pandas as pd 
from sklearn.feature_extraction import DictVectorizer 
from sklearn.ensemble import RandomForestRegressor 
from sklearn.metrics import mean_squared_error
from sklearn.metrics import root_mean_squared_error
from sklearn.pipeline import make_pipeline

In [4]:
import mlflow

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("green-taxi-duration")

<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1748130657517, experiment_id='1', last_update_time=1748130657517, lifecycle_stage='active', name='green-taxi-duration', tags={}>

In [6]:
# Read dataframe 
def read_dataframe(filename: str):
    "Wrtie a function to read dataframe"
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df


def prepare_dictionaries(df: pd.DataFrame):
    "Write a function to prepare dictionaries"
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient = 'records')
    return dicts 

In [8]:
df_train = read_dataframe('../../00-Dataset/green_tripdata_2021-01.parquet')
df_val = read_dataframe('../../00-Dataset/green_tripdata_2021-02.parquet')

target = 'duration'
y_train = df_train[target].values 
y_val = df_val[target].values 

dict_train = prepare_dictionaries(df_train)
dict_val = prepare_dictionaries(df_val)

In [10]:
with mlflow.start_run(run_name="Random Forest Regressor"):
    # Log tags for developer and model
    mlflow.set_tag("developer", "Dario")
    mlflow.set_tag("model_name", "random_forest")
    
    params = dict(max_depth=20, n_estimators=100, min_samples_leaf=10, random_state=0)
    mlflow.log_params(params)
    
    dv = DictVectorizer()

    pipeline = make_pipeline(
        dv,
        RandomForestRegressor(**params, n_jobs=-1)
    )

    pipeline.fit(dict_train, y_train)
    y_pred = pipeline.predict(dict_val)

    rmse = root_mean_squared_error(y_pred, y_val)
    print(params, rmse)
    mlflow.log_metric('rmse', rmse)

    mlflow.sklearn.log_model(pipeline, artifact_path="model")

    with open('dict_vectorizer.bin', 'wb') as f_out:
        pickle.dump(dv, f_out)
        
    mlflow.log_artifact('dict_vectorizer.bin')

{'max_depth': 20, 'n_estimators': 100, 'min_samples_leaf': 10, 'random_state': 0} 6.7558229919200725


2025/05/26 12:10:33 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Random Forest Regressor at: http://127.0.0.1:5000/#/experiments/1/runs/33d91be4d8184963b8648d4419ef6507
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


In [12]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = 'http://127.0.0.1:5000/'
RUN_ID = '33d91be4d8184963b8648d4419ef6507'
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [14]:
path = client.download_artifacts(run_id = RUN_ID, path='dict_vectorizer.bin')

In [16]:
with open(path, 'rb') as f_out:
    dv = pickle.load(f_out)

In [18]:
dv

DictVectorizer()